You are in charge of developing a framework to detect fraud for a life insurance company. The life
insurance company services many the entire nation, but you will only be working with customer data
from their west coast branches. The company has never done any fraud modeling in the past, so you will
be starting from scratch.  
Now that you have received your results from the “domain expert” about your unsupervised learning
techniques, you can build a model to try and predict “fraud”. You are welcome to use any modeling
approach you want (or any combination of them). Your assignment is to turn in a rank ordering of the
500 claims that are most likely to be fraudulent. The team with the highest number of actual fraudulent
cases as well as the most money saved (weighted average of ranks) in these will win the prize! Email this
list to adlabarr@ncsu.edu. 

In [57]:
import pandas as pd
import numpy as np
import xgboost as xgb

joining our teams findings bc we had the highest correct proprtion

In [58]:
df = pd.read_csv('aggregated_policy_label_2025.csv')
df_ours = pd.read_csv('blue-14-assignment-2.csv')
df_ours['Susp_fraud_Ours'] = 1
df_combo = pd.merge(df, df_ours, how='left', on='Cust_ID')
df_combo['Susp_fraud_Ours'] = df_combo['Susp_fraud_Ours'].fillna(0)
df_combo['total_frauds'] = (df_combo[['Susp_fraud_Ours', 'Susp_fraud_Both']] == 1).any(axis=1).astype(int)
df_combo

,GivenName,MiddleInitial,Surname,StreetAddress,City,State,ZipCode,Country,TelephoneNumber,MothersMaiden,...,Zip_per_CustID,Avg_CL,PerSize_Max_CH,SD_CovLim_Last_6mon,Time_MaritalCH_CL,Susp_fraud_Orange,Susp_fraud_Blue,Susp_fraud_Both,Susp_fraud_Ours,total_frauds
0,James,M,Baker,1782 Freedom Lane,Atwater,CA,95301,US,209-356-7271,McDonald,...,1,50000,0.0,0.0,NaN,0,0,0,0.0,0
1,Melanie,C,Johnson,3230 Gateway Avenue,Lancaster,CA,93534,US,661-945-1609,Johnson,...,1,100000,0.0,0.0,NaN,0,0,0,0.0,0
2,Anne,T,Bailey,1011 Maple Street,Anaheim,CA,92801,US,714-330-5620,Lehoux,...,1,100000,0.0,0.0,0.0,0,0,0,0.0,0
3,Geoffrey,E,Veiga,2370 Roosevelt Street,San Francisco,CA,94103,US,415-436-5690,Johnson,...,1,50000,0.0,0.0,NaN,0,0,0,0.0,0
4,Sylvia,W,Hines,4404 Lindale Avenue,Oakland,CA,94607,US,510-551-8237,Woodward,...,1,150000,0.0,0.0,NaN,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44483,Thelma,J,Ortega,443 Zimmerman Lane,Los Angeles,CA,90057,US,213-487-0348,Lucas,...,1,150000,0.0,0.0,NaN,0,0,0,0.0,0
44484,Janice,E,Murphy,3285 Edsel Road,Los Angeles,CA,90017,US,818-444-6525,Godfrey,...,1,100000,0.0,0.0,NaN,0,0,0,0.0,0
44485,Joel,S,Landis,1131 Cemetery Street,Salinas,CA,93901,US,831-784-8817,Clayton,...,1,50000,0.0,0.0,NaN,0,0,0,0.0,0
44486,Betty,F,Carson,2302 Hall Street,Las Vegas,NV,89121,US,702-435-8435,O'Donnell,...,1,150000,0.0,0.0,NaN,0,0,0,0.0,0


## Data Prep

Doing the same exact thing that I did for the last assignment

In [59]:
junk = ['GivenName', 'MiddleInitial', 'Surname', 'StreetAddress', 'City', 'State',
    'ZipCode', 'Country', 'TelephoneNumber', 'MothersMaiden', 'Birthday',
    'BloodType', 'Pounds', 'FeetInches', 'Date_Initial', 
    'Cov_ID', 
    'Susp_fraud_Ours', 'Susp_fraud_Both', 'Susp_fraud_Blue', 'Susp_fraud_Orange']
df_combo = df_combo.drop(columns=junk)

missing_more_50_per = ['Time_LastChange_Claim', 'Marriage', 'Time_MaritalCH_CL']
df_combo = df_combo.drop(columns=missing_more_50_per)

remaning_cols = ['Adj_ZIP', 'Distance_Claim_Adj', 'Pop_CZIP', 'MedInc_CZIP']
df_combo = df_combo.drop(columns=remaning_cols)

# set the customer idf as index
df_combo.set_index('Cust_ID', inplace=True)

# replace all 'Y' values with 1 and all 'N' values with 0
df_combo.replace({'Y': 1, 'N': 0}, inplace=True) 

# For compute times lets drop columns with more than 50 unique categories - ik this isnt best practice but this assignment is rushed
df_combo.drop(
    columns=[col for col in df_combo.select_dtypes(include=['object']).columns if df[col].nunique() > 50],
    inplace=True
)

df_combo = pd.get_dummies(df_combo, drop_first=False)

C:\Users\Sterling Hayden\AppData\Local\Temp\ipykernel_22600\1053596585.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_combo.replace({'Y': 1, 'N': 0}, inplace=True)


fraud is a rare event, lets fix that

In [60]:
# # prop of fraudsters
# print(sum(df_combo.total_frauds)/df.shape[0])

# train = df_combo.groupby("total_frauds").sample(n = 1200, random_state=12345)
# test = df_combo[~df_combo['total_frauds'].isin(train.index)]

# # new p
# print(sum(train.total_frauds)/train.shape[0])

Now, we need to weight our observations. If our models do a good job at predicting this training set, it might not be as good as we think. This is due to the training set not looking like our population. When building models, we can weight our observations so that the predictions coming from the model are adjusted back to the original population proportion of our target.

In [61]:
# p_0 = 0.5 
# p_1 = 0.5
# P_1 = sum(df_combo.total_frauds)/df.shape[0]
# P_0 = 1 - P_1

# train['weight'] = np.where(train['total_frauds'] == 1, 1, (p_1*P_0)/(p_0*P_1))

# pd.crosstab(train['total_frauds'], train['weight']) 

## Model Building Time

In [62]:
X = df_combo.drop(columns=['total_frauds'])
y = df_combo['total_frauds']

# Define Model with Class Imbalance Handling
fit_xgb = xgb.XGBClassifier(
    max_depth=3,
    n_estimators=500,
    objective='binary:logistic',  # Predicts probability scores
    scale_pos_weight=sum(df_combo.total_frauds)/df.shape[0], # handles imbalance data insted of doing smote
    eval_metric='logloss'
)

# Train Model
sample_weights = 1 + 0.1 * X['Reward_Amount'] # gives higher weight to samples with larger Reward_Amount
sample_weights = np.clip(sample_weights, -10, 20)  # Limit weights between 1 and 10
fit_xgb.fit(X, y, sample_weight=sample_weights)

# Predict Probabilities
df_combo['pred_xgb'] = fit_xgb.predict_proba(X)[:,1]

In [63]:
top_500 = df_combo['pred_xgb'].sort_values(ascending=False).head(500)
top_500.to_csv('blue-14-assignment-3.csv')
top_500

Cust_ID
PSX000817530    1.000000
PSX000287942    1.000000
PSX000819504    1.000000
PSX000965870    1.000000
PSX000616487    1.000000
                  ...   
PSX000415993    0.999579
PSX000782493    0.999574
PSX000500187    0.999574
PSX000412948    0.999563
PSX000802237    0.999561
Name: pred_xgb, Length: 500, dtype: float32